In [1]:
import os
import numpy as np
import random
import cv2
import gc

# Ana dizin
root_dir = 'dataset'

# Çıktı dosyaları
output_images_file = 'selected_images.npy'
output_labels_file = 'selected_labels.npy'

random.seed(42)

# Klasörleri al
class_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
if '__pycache__' in class_folders:
    class_folders.remove('__pycache__')
class_folders.sort()
print(class_folders)
selected_images = []
selected_labels = []


def loadNp(file_path , label):
    data = np.load(file_path)  # shape: [slice, H, W]
    print(data.dtype)
    print(f"İşlenen Dosya: {file_path}, shape: {data.shape}")
    

    ver_cent = data.shape[1]//2
    hor_cent = data.shape[2]//2

    size = 240//2
    
    data = data[:,ver_cent-size:ver_cent+size,hor_cent-size:hor_cent+size]


    slice_count = data.shape[0]
    slice_indices = random.sample(range(slice_count), min(500, slice_count))
    print(len(slice_indices))
    selected_images.extend(data[slice_indices])
    for i in range(500):
        selected_labels.append(label)
    gc.collect()
    del data

for label, folder in enumerate(class_folders):
    folder_path = os.path.join(root_dir, folder)
    npy_files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

    for file in npy_files:
        file_path = os.path.join(folder_path, file)
        loadNp(file_path=file_path,label=label)
        
# Listeyi numpy array'e çevir
selected_images = np.array(selected_images)
selected_labels = np.array(selected_labels)

# Kaydet
np.save(output_images_file, selected_images)
np.save(output_labels_file, selected_labels)

print(f"Toplam {len(selected_images)} slice ve {len(selected_labels)} etiket kaydedildi.")



['tcga_astro_filtered', 'tcga_gbm_filtered', 'tcga_olio_filtered']
uint8
İşlenen Dosya: dataset/tcga_astro_filtered/TCGA-P5-A737_30.npy, shape: (63810, 240, 240)
500


KeyboardInterrupt: 

In [ ]:
def process_dcm_file(dcm_path, save_dir):
    try:
        # Read using force to speed up + stop after pixel data
        ds = pd.dcmread(dcm_path, force=True, stop_before_pixels=False)

        # Decode JPEG2000 more efficiently (if applicable)
        arr = ds.pixel_array  # Pydicom will use pylibjpeg/gdcm if installed

        # Optional: apply VOI LUT if necessary
        # arr = apply_voi_lut(arr, ds)

        # Build output path
        pid = os.path.basename(dcm_path)[:17]
        
        h = ds.TotalPixelMatrixColumns
        w = ds.TotalPixelMatrixRows
        
        output_path = os.path.join(save_dir, pid +"(" + str(w)+"x" + str(h) + ")" + ".npy")

        # Save as NumPy file
        np.save(output_path, arr)

    except Exception as e:
        print(f"Failed to process {dcm_path}: {e}")

In [ ]:
for folder in folders:
    print(f"Processing folder: {folder}")  
    
    if folder == "tcga_olio_filtered":
        label = 0
    elif folder == "tcga_astro_filtered":
        label = 1
    elif folder == "tcga_gbm_filtered":
        label = 2
    
    path = os.path.join("./dataset2", folder)
    del_and_crt_dir(path)

    # Use ThreadPoolExecutor to process files in parallel
    dcm_files = glob.glob(os.path.join(folder, "*.dcm"))
    
    with ThreadPoolExecutor(max_workers=2) as executor:
        executor.map(lambda file: process_dcm_file(file, path), dcm_files)


Processing folder: tcga_olio_filtered
Processing folder: tcga_astro_filtered


In [4]:
expected_shape = (240, 240)  # Veya gri ise (240, 240)

for i, img in enumerate(selected_images):
    if img.shape != expected_shape:
        print(f"Image {i} has shape: {img.shape}")

In [5]:

resized_images = [cv2.resize(img, (240, 240)) for img in selected_images]
np_images = np.array(resized_images)

In [6]:

np_labels = np.array(labels)

np.save('images.npy', np_images)  
np.save('labels.npy', np_labels)              


In [7]:

# path = "TCGA-R8-A6MK_93789065-5d39-4725-af39-41e3c23a2289.dcm"
# process_dcm_file(path, "./")